# The dataset
1. The dataset has three splits:
    + train
    + test
    + unsupervised
   Splits can be found in the hugging faces page of the dataset.
   Or with the function get_dataset_split_names("name_of_dataset")
2. Here are the size of datasets:
    + Size of the train dataset: 25000
    + Size of the test dataset: 25000
    + Size of the unsupervised dataset: 50000

In [ ]:
from datasets import Dataset
from datasets import load_dataset
from datasets import load_dataset_builder

import math

In [ ]:
dataset = load_dataset_builder("imdb")
dataset_train = load_dataset("imdb", split='train')
dataset_test = load_dataset("imdb", split='test')
dataset_unsupervised = load_dataset("imdb", split='unsupervised')

In [ ]:
dataset.info.description

In [ ]:
print("Size of the train dataset: " + str(len(dataset_train)))
print("Size of the test dataset: " + str(len(dataset_test)))
print("Size of the unsupervised dataset: " + str(len(dataset_unsupervised)))

# Naive Bayes classifier

## I. Preprocessing

In [ ]:
punctuation_filter = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+',
                      ',', '.', '/', ':', ';', '<', '=', '>', '?', '@',
                      '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']

def to_lower_case(row: dict) -> dict:
    """
    Lower text field in the row dict
    return: updated row
    """
    row['text'] = row['text'].lower()
    return row

def remove_punctuation(row: dict) -> dict:
    """
    Replace punctuation from punctuation_filter list to
    spaces in the text field of row dict
    return: updated row
    """
    for punctuation in punctuation_filter:
        row['text'] = row['text'].replace(punctuation, ' ')
    return row

def preprocessing(row: dict) -> dict:
    """
    Lower text field in the row dict and replace punctuation
    from punctuation_filter list to spaces in the text field
    of row dict
    return: updated row
    """
    return to_lower_case(remove_punctuation(row))

In [ ]:
preprocess_train = dataset_test.map(preprocessing)
preprocess_test = dataset_train.map(preprocessing)
preprocess_unsupervised = dataset_unsupervised.map(preprocessing)

## II. Naive Bayes classifier

### Our implementation

In [ ]:
def train_naive_bayes(documents: Dataset, classes: list):
    logprior = {}
    loglikelihood = {k: {} for k in classes}
    
    # Vocabulary of documents
    voc = {} # Histogram {word: count}
    class_voc = {k: {} for k in classes}
    total_count = 0
    
    def update_voc(document: Dataset) -> None:
        if isinstance(document['text'], str):
            words = document['text'].split()
        else:
            words = document['text']    
        nonlocal total_count
        total_count += len(words)
        for word in words:
            voc.update({word: voc.get(word, 0) + 1})
            c = document['label']
            class_voc[c].update({word: class_voc[c].get(word, 0) + 1})
    
    documents.map(update_voc)

    # Update total count for loglikelihood formula
    total_count += len(voc)

    for c in classes:
        num_doc = len(documents)
        c_docs = documents.filter(lambda doc: doc['label'] == c)
        num_c = len(c_docs)
        logprior[c] = math.log(num_c / num_doc)

        for word in voc.keys():
            loglikelihood[c][word] = math.log((class_voc[c].get(word, 0) + 1)/total_count)
        
    return logprior, loglikelihood, voc

In [ ]:
def test_naive_bayes(test_str: str, logprior: dict, loglikelihood: dict, classes: list, voc: dict) -> int:
    sum_max = None
    c_max = None
    for c in classes:
        sum_c = logprior[c]
        for word in test_str.split():
            if word in voc:
                sum_c += loglikelihood[c][word]
        if not sum_max or sum_max < sum_c:
            sum_max = sum_c
            c_max = c
    return c_max

In [ ]:
classes = [0, 1]
logprior, loglikelihood, voc = train_naive_bayes(preprocess_train, classes)

In [ ]:
def accuracy(preprocess_test: Dataset, logprior: dict, loglikelihood: dict, classes: list, voc: dict):
    confusion = [0, 0, 0, 0] # TP, TN, FP, FN
    accuracy=0
    def update_voc(document: Dataset) -> None:
        nonlocal accuracy
        if isinstance(document['text'], str):
            comment = document['text']
        else:
            comment = " ".join(document['text'])
        res = test_naive_bayes(comment, logprior, loglikelihood, classes, voc)
        confusion[1-res + (2 * (1-document['label']))] += 1
        accuracy += res == document['label']
    preprocess_test.map(update_voc)
    accuracy = accuracy/len(preprocess_test)
    return accuracy

In [ ]:
print('Our implementation accuracy:', accuracy(preprocess_test, logprior, loglikelihood, classes, voc))

### Scikit
#### 3. Implement a naive Bayes classifier using scikit-learn.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(analyzer = "word", 
                             lowercase=True, 
                             tokenizer = None, 
                             preprocessor = None, 
                             stop_words = None, 
                             max_features = 5000)
db = preprocess_train
y = db['label']
corpus = db['text']
bag_of_words = vectorizer.fit_transform(corpus)
print(bag_of_words.shape)
X = bag_of_words.toarray()

clf = MultinomialNB(force_alpha=True)
clf.fit(X, y)

In [ ]:
#vectorizer.get_feature_names_out()
#print ((clf.feature_log_prob_))
vocabulary = vectorizer.vocabulary_
#print(vocabulary)
vectorizer.transform(['love in my ass'])

In [ ]:
print(clf.predict(vectorizer.transform(['I love this movie'])))

In [ ]:
y = preprocess_test['label']
X = vectorizer.transform(preprocess_test['text'])
score = clf.score(X, y)
print('Scikit learn accuracy:', score)

#### 4. Report the accuracy on both training and test set, for both your implementation and the scikit-learn one.
- Our implementation accuracy: 0.82152
- Scikit learn accuracy: 0.83076


#### 5. Most likely, the scikit-learn implementation will give better results. Looking at the documentation, explain why it could be the case.
Scikit give results that are 1% better, which might be due to being a libarary. In general, libraries are well optimized.

#### 6. Why is accuracy a sufficient measure of evaluation here?
Accuracy is 


#### 7. Using one of the implementation, take at least 2 wrongly classified example from the test set and try explaining why the model failed.

# Stemming and Lemmatization

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy

# loading the small English model
nlp = spacy.load("en_core_web_sm")

In [ ]:
def lemmatization(row: dict) -> dict:
    """
    do the lemmatization
    """
    row['text'] = [token.lemma_ for token in nlp(row['text'])]
    return row

In [ ]:
lemmatization_train = preprocess_train.map(lemmatization)

In [ ]:
lemmatization_test = preprocess_test.map(lemmatization)

In [ ]:
classes = [0, 1]
logprior, loglikelihood, voc = train_naive_bayes(lemmatization_train, classes)

In [ ]:
print(accuracy(lemmatization_test, logprior, loglikelihood, classes, voc))